In [2]:
import param

In [ ]:
param.parameterized.P

In [3]:
# default_exp algo.dl.torchtext

%reload_ext autoreload
%autoreload 2

# algo-dl-torchtext

https://pytorch.org/text/

https://github.com/pytorch/text

https://github.com/atnlp/torchtext-summary

The torchtext package consists of 
* data processing utilities   
对应的代码封装在 `torchtext.data`: Generic data loaders, abstractions, and iterators for text (including vocabulary and word vectors)
* popular datasets for natural language.    
`torchtext.datasets`: Pre-built loaders for common NLP datasets

## install

In [4]:
# !pip install torchtext -U
!pip freeze | grep torchtext

torchtext==0.7.0


# torchtext.data
## 数据抽象
### Example级别
* Dataset, TabularDataset, Batch:   
Defines a dataset composed of Examples
* Example  
Defines a single training or test example.

### Field级别
主要的操作都在Field级别

数据字段可以划分为两类: 序列数据和非序列数据。

针对序列数据，常见的处理方式有: 
* 分词 
* 填补至等长pad 
* 截断trucate

## 常见的序列Field处理方式

In [1]:
from torchtext import data

### 超级棒的封装，基本上囊括了所有的情况

In [ ]:
data.Field(
    sequential=True,  # Whether the datatype represents sequential data. If False,
                        # no tokenization is applied. Default: True.
    use_vocab=True,  
    init_token=None,# A token that will be prepended to every example using this
                        # field, or None for no initial token. Default: None.
    eos_token=None,
    fix_length=None,  #  A fixed length that all examples using this field will be
                        # padded to, or None for flexible sequence lengths. Default: None.
    dtype=torch.int64,
    preprocessing=None,  #  The Pipeline that will be applied to examples
                            # using this field after tokenizing but before numericalizing. Many
                            # Datasets replace this attribute with a custom preprocessor.
    postprocessing=None,
    lower=False,
    tokenize=None,  # The function used to tokenize strings using this field into
                    # sequential examples. If "spacy", the SpaCy tokenizer is
                    # used. If a non-serializable function is passed as an argument,
                    # the field will not be able to be serialized. Default: string.split.
    tokenizer_language='en',
    include_lengths=False,  # Whether to return a tuple of a padded minibatch and
                            # a list containing the lengths of each examples, or just a padded
                            # minibatch. Default: False.
    batch_first=False,
    pad_token='<pad>',
    unk_token='<unk>',
    pad_first=False,
    truncate_first=False,
    stop_words=None,
    is_target=False,  # Whether this field is a target variable.
                        # Affects iteration over batches. Default: False
)

### 序列处理

In [67]:
TEXT=data.Field(sequential=True, 
#                 tokenize=tokenize,
                lower=True,
                batch_first=False,  # 注意，默认False，即处理后的结果是(seq_len, batch_size, embed_dim)
                fix_length=3
               )


### label字段处理

In [ ]:
LABEL=data.Field(sequential=False, use_vocab=False)

## Iterators

### Iterator

In [ ]:
data.Iterator(
    dataset,
    batch_size,
    sort_key=None,
    device=None,  # (str or `torch.device`): A string or instance of `torch.device`
                    # specifying which device the Variables are going to be created on.
                    # If left as default, the tensors will be created on cpu. Default: None.
    batch_size_fn=None,
    train=True,
    repeat=False,
    shuffle=None,
    sort=None,  # A key to use for sorting examples in order to batch together
                # examples with similar lengths and minimize padding. The sort_key
                # provided to the Iterator constructor overrides the sort_key
                # attribute of the Dataset, or defers to it if None.
    sort_within_batch=None,  # Whether to sort (in descending order according to
                            # self.sort_key) within each batch. If None, defaults to self.sort.
                            # If self.sort is True and this is False, the batch is left in the
                            # original (ascending) sorted order.
)

### BucketIterator

In [ ]:
data.BucketIterator(
    dataset,
    batch_size,
    sort_key=None,
    device=None,
    batch_size_fn=None,
    train=True,
    repeat=False,
    shuffle=None,
    sort=None,
    sort_within_batch=None,
)

## 示例

### 示例1
https://www.yanxishe.com/blogDetail/14658

In [68]:
# 分词函数
def content_tokenize(text):    
    return [item for item in str(text)]
# 创建content字段的Field
CONTENT = data.Field(sequential=True, tokenize=content_tokenize, batch_first=True)
# 创建label字段的
FieldLABEL = data.Field(sequential=False, use_vocab=False)


In [77]:
fields = [('label', LABEL), ('content', CONTENT)]
examples = []
for label, content in [(1, '我不退我也不买。我就看戏'), (0, '我今天很开心，很开心，微信'), 
                      (1, '跟诸葛撞单了')]:    
    examples.append(data.Example.fromlist([label, content], fields))
data_set = data.Dataset(examples, fields)

In [82]:
data_set.examples.index

<function list.index(value, start=0, stop=9223372036854775807, /)>

#### build_vocab

In [72]:
# 为 CONTENT 字段创建词向量
CONTENT.build_vocab(data_set)

In [81]:
CONTENT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x14c86cb00>>,
            {'<unk>': 0,
             '<pad>': 1,
             '我': 2,
             '不': 3,
             '开': 4,
             '很': 5,
             '心': 6,
             '，': 7,
             '。': 8,
             '也': 9,
             '买': 10,
             '了': 11,
             '今': 12,
             '信': 13,
             '单': 14,
             '天': 15,
             '就': 16,
             '微': 17,
             '戏': 18,
             '撞': 19,
             '看': 20,
             '葛': 21,
             '诸': 22,
             '跟': 23,
             '退': 24})

In [80]:
CONTENT.vocab.itos

['<unk>',
 '<pad>',
 '我',
 '不',
 '开',
 '很',
 '心',
 '，',
 '。',
 '也',
 '买',
 '了',
 '今',
 '信',
 '单',
 '天',
 '就',
 '微',
 '戏',
 '撞',
 '看',
 '葛',
 '诸',
 '跟',
 '退']

In [76]:
CONTENT.vocab.freqs

Counter({'我': 4,
         '不': 2,
         '退': 1,
         '也': 1,
         '买': 1,
         '。': 1,
         '就': 1,
         '看': 1,
         '戏': 1,
         '今': 1,
         '天': 1,
         '很': 2,
         '开': 2,
         '心': 2,
         '，': 2,
         '微': 1,
         '信': 1,
         '跟': 1,
         '诸': 1,
         '葛': 1,
         '撞': 1,
         '单': 1,
         '了': 1})

### 示例2 tx

In [ ]:
# TEXT = data.Field(sequential=True, fix_length=50, batch_first=True)
# LABEL = data.Field(sequential=False, use_vocab=False, batch_first=True)

TEXT = data.Field(sequential=True, fix_length=50)
LABEL = data.Field(sequential=False, use_vocab=False)

In [ ]:
# 定义Dataset
class MyDataset(data.Dataset):
    name = 'TX Dataset'

    @staticmethod
    def sort_key(ex):
        return len(ex.creative_ids)

    def __init__(self, path, text_field, label_field, test=False, aug=False, **kwargs):
        fields = [("id", None), # we won't be needing the id, so we pass in None as the field
                 ("creative_ids", text_field), ("gender", label_field)]
        
        examples = []
        df = pd.read_pickle(path)
        print('read data from {}'.format(path))

        if test:
            # 如果为测试集，则不加载label
            for text in tqdm(csv_data['comment_text']):
                examples.append(data.Example.fromlist([None, text, None], fields))
        else:
            for text, label in tqdm(zip(df['creative_id'], df['gender'])):
                if aug:
                    # do augmentation
                    rate = random.random()
                    if rate > 0.5:
                        text = self.dropout(text)
                    else:
                        text = self.shuffle(text)
                # Example: Defines a single training or test example.Stores each column of the example as an attribute.
                examples.append(data.Example.fromlist([None, text, label], fields))
        # 之前是一些预处理操作，此处调用super调用父类构造方法，产生标准Dataset
        # super(MyDataset, self).__init__(examples, fields, **kwargs)
        super(MyDataset, self).__init__(examples, fields)

    def shuffle(self, text):
        text = np.random.permutation(text.strip().split())
        return ' '.join(text)

    def dropout(self, text, p=0.5):
        # random delete some text
        text = text.strip().split()
        len_ = len(text)
        indexs = np.random.choice(len_, int(len_ * p))
        for i in indexs:
            text[i] = ''
        return ' '.join(text)


In [ ]:
train = MyDataset(
    os.path.join(data_dir, 'train_gender.pkl'), 
    text_field=TEXT, 
    label_field=LABEL, 
    test=False, 
    aug=False)

val = MyDataset(
    os.path.join(data_dir, 'val_gender.pkl'), 
    text_field=TEXT, 
    label_field=LABEL, 
    test=False, 
    aug=False)

In [ ]:
train[0].__dict__.keys()

In [ ]:
TEXT.build_vocab(train)

#### Iterator

In [ ]:
from torchtext.data import Iterator, BucketIterator

In [ ]:
train_iter, val_iter = BucketIterator.splits(
        (train, val), # 构建数据集所需的数据集
        batch_sizes=(100, 100),
#         device=-1, # 如果使用gpu，此处将-1更换为GPU的编号
        # device='cpu',
        # sort_key=lambda x: len(x.creative_ids), # the BucketIterator needs to be told what function it should use to group the data.
        # sort_within_batch=False,
        # repeat=False # we pass repeat=False because we want to wrap this Iterator layer.
)

In [ ]:
for idx, batch in enumerate(train_iter):
    print(batch)
    text, label = batch.creative_ids, batch.gender
    # print(text.shape, label.shape)
    print(text)
    print(label)
    break

## Text Classification with TorchText
https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

This tutorial shows how to use the text classification datasets in torchtext, including

- AG_NEWS,
- SogouNews,
- DBpedia,
- YelpReviewPolarity,
- YelpReviewFull,
- YahooAnswers,
- AmazonReviewPolarity,
- AmazonReviewFull



In [1]:
#export


In [ ]:
from nbdev.export import notebook2script
notebook2script('.ipynb')